In [1]:
import pickle
import difflib
from tqdm.autonotebook import tqdm

/Users/maxim/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
SOS_token = 0
EOS_token = 1


class Lang(object):
    def __init__(self, name, n=5, threshold=10):
        self.name = name
        self.word2int = {}
        self.word2count = {}
        self.int2count = {}
        self.int2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.n_words = 2
        self.n = 5
        self.threshold = threshold

    def addWord(self, word):
        if word not in self.word2int:
            self.word2int[word] = self.n_words
            self.word2count[word] = 1
            self.int2count[self.n_words] = 1
            self.int2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            self.int2count[self.word2int[word]] += 1

    def addSentence(self, sentence):
        words = sentence.split(" ")
        for word in words:
            self.addWord(word)

    def word2valid(self, word):
        i = self.word2int[word]
        if i in self.valid:
            return self.valid2int[i]
        else:
            print('Not valid')
            return 0

    def valid2word(self, v):
        return self.int2word[self.int2valid[v]]

    def addText(self, sentences):
        for sentence in sentences:
            self.addSentence(sentence)
        tmp = [np.array([0]), np.array([0])]
        for i in range(2, self.n_words):
            tmp.append(
                np.zeros(self.word2count[self.int2word[i]] * self.n * 2,
                         dtype=np.int))
        self.int2context = np.array(tmp)
        tmp = np.zeros(len(tmp), dtype=np.int)
        for sentence in tqdm(sentences):
            words = sentence.split(" ")
            words = [self.word2int[i] for i in words]
            for i, w in enumerate(tqdm(words, leave=False)):
                if self.word2count[self.int2word[w]] > self.threshold:
                    for j in range(max(i - self.n, 0), i):
                        self.int2context[w][
                            tmp[w]] = words[j] if self.int2count[
                                words[j]] > self.threshold else 0
                        tmp[w] += 1
                    for j in range(i + 1, min(len(words), i + 1 + self.n)):
                        self.int2context[w][
                            tmp[w]] = words[j] if self.int2count[
                                words[j]] > self.threshold else 0
                        tmp[w] += 1
        for i in range(2, self.n_words):
            self.int2context[i] = self.int2context[i][self.int2context[i] != 0]
        self.valid = []
        for i in range(2, self.n_words):
            if self.int2count[i] > self.threshold:
                self.valid.append(i)
        self.n_valid = len(self.valid)
        self.int2valid = {}
        self.valid2int = {}
        for i, v in enumerate(self.valid):
            self.valid2int[v] = i
            self.int2valid[i] = v
        self.valid2context = self.int2context[self.valid]
        for i in tqdm(range(len(self.valid2context))):
            for j in tqdm(range(len(self.valid2context[i])), leave=False):
                self.valid2context[i][j] = self.valid2int[self.valid2context[i]
                                                          [j]]

In [3]:
f = open('lang.pkl', 'rb')
lang = pickle.load(f)

In [7]:
lang.word2int("hermione")

TypeError: 'dict' object is not callable

In [4]:
from rewrite import rewrite

In [5]:
mod = rewrite(lang, path2dict="enwiki_2017_08_20_trimmed.dict")

In [6]:
ans = mod.get_sim()

In [7]:
ans[:10]

[['suckin', 'sucking', 0.9230769230769231],
 ['prefrerred', 'preferred', 0.9473684210526315],
 ['sanything', 'anything', 0.9411764705882353],
 ['corrrect', 'correct', 0.9333333333333333],
 ['forenoons', 'forenoon', 0.9411764705882353],
 ['alrigh', 'alright', 0.9230769230769231],
 ['soounds', 'sounds', 0.9230769230769231],
 ['leter', 'letter', 0.9090909090909091],
 ['piercer', 'pierce', 0.9230769230769231],
 ['tcickets', 'tickets', 0.9333333333333333]]

In [8]:
with open("rewrite.pkl", "wb") as fout:
    pickle.dump(mod, fout)

In [9]:
with open("./data/data.txt", "r") as fout:
    data = fout.read()

In [10]:
data = rewrite.clean_text(data, ans)

In [11]:
with open("./data/data.txt", "w") as fout:
    fout.write(data)

In [15]:
with open("./data/kaggle_text_2_prep.txt", "r") as fout:
    data = fout.read()

In [16]:
data = rewrite.clean_text(data, ans)
with open("./data/kaggle_text_2_prep.txt", "w") as fout:
    fout.write(data)